#explanatory

This cell imports the necessary libraries we will need for this demonstration

* **random** is imported for ...
* **mesa** imports core mesa to give us basic agentbased model functionality
* **mesa_geo** imports GIS functionality for agent based models
* **mesa_geo.experimental** is a sub-module of `mesa_geo` that will will import as mge so we do not have to write out `mesa_geo.experimental` everytime

# Introductory Tutorial

This tutorial introduces Mesa-Geo, a GIS integrated Agent Based model platform that is part of the [Mesa ABM Ecosystem](https://mesa.readthedocs.io/en/stable/) 

In this tutorial we will build a Geo-Schelling model, which takes the seminal [Schelling Segregation model](https://en.wikipedia.org/wiki/Schelling%27s_model_of_segregation) and replicates with the countries of Europe, were the countries "move" their preference similar to the Schelling Model. 

This tutorial also uses [Jupyter Bridge](https://pypi.org/project/jupyter-bridge/) so users can see more detailed explanations of each code cell if desired by clicking the book icon. 

#explanatory 

This is only needed if running in Google Colab as mesa_geo is not part of Googles standard build

In [1]:
# Run this cell if in Google Colab!
!pip install mesa_geo --quiet

#explanatory

For the import cell we use three libraries: 

* **random** this is a standard python library and used in this model to randomize the country (agents) behavior and preferences
* **mesa** this is the core ABM library that provides basic ABM functionality for `mesa-geo`
* **mesa_geo** this library add GIS features to Mesa to allow for GIS integration into Agent Based Models
* **mesa_geo.visualization** is a sub_module of mesa_geo, we import this module to make the code more concise so we we do not have to type mg.visualization every time we use this visualization module

In [2]:
import random

import mesa 
import mesa_geo as mg
import mesa_geo.visualization as mgv

## Create the Agents

The next cell creates the GeoAgents, in which European country is randomly assigned a preference of minority or majority and then if the agent is unhappy moves.  

#explanatory

**Agent Class and Initialization** 
To create numerous agents with  different attributes, we instantiate a Agent class (i.e. Schelling Agent), this is the common Mesa approach and it inherits from Mesa-Geo's GeoAgent class.

The GeoAgent always has four attributes: 
1. The unique ID
2. The model so the agents can reference it
3. The geometry
4. The projection or crs

In this simulation there is one additonal attribute agent_type which identifies if the agent is a minority or majority agents.

**Step Function**
The step function is Mesa syntax and required for each agent. In the model class, discussed next, the step function for each agent is called by the model step function and represents what each agent does for each time step. 

In this case the agent first identifies all its neighbors and determines if they are the same (minority or majority) if the neighbors who are not the same type as the agent are greater than those who are similar than the agent moves. 

If the number of neighbors who are similar is greater than those who are different than agent is happy and does not move. 

In [3]:
class SchellingAgent(mg.GeoAgent):
    """Schelling segregation agent."""

    def __init__(self, unique_id, model, geometry, crs, agent_type=None):
        """Create a new Schelling agent.

        Args:
            unique_id: Unique identifier for the agent.
            agent_type: Indicator for the agent's type (minority=1, majority=0)
        """
        super().__init__(unique_id, model, geometry, crs)
        self.atype = agent_type

    def step(self):
        """Advance agent one step."""
        similar = 0
        different = 0
        neighbors = self.model.space.get_neighbors(self)
        if neighbors:
            for neighbor in neighbors:
                if neighbor.atype is None:
                    continue
                elif neighbor.atype == self.atype:
                    similar += 1
                else:
                    different += 1

        # If unhappy, move:
        if similar < different:
            # Select an empty region
            empties = [a for a in self.model.space.agents if a.atype is None]
            # Switch atypes and add/remove from scheduler
            new_region = random.choice(empties)
            new_region.atype = self.atype
            self.model.schedule.add(new_region)
            self.atype = None
            self.model.schedule.remove(self)
        else:
            self.model.happy += 1

    def __repr__(self):
        return "Agent " + str(self.unique_id)

## Model Class

This class initiates the model class, which acts as a "manager" for the simulation, it holds the geo-spatial space. It holds the schedule of agents and their order, and the data collector to collect information from the model.

#explanatory

The model class initiates the "manager" for the simulation. Initiatiang the initial parameters and inheriting core attributes from Mesa's models class. In this case there are three parameters, density of agents (how many of the existing countries are agents), what percent are minority, and whether or not to export the data. 

Then there are three main parts: 
1. The schedule which in this case uses random activation, this is just a list of agent objects, who's order is randomly reordered after each step.
2. The space which uses geomesa space module and has a keyword argument of the projection conversion
3. The datacollector which in this case collects the number of agent who are happy with their location (neighbors are more similar than different)

The model class step function then calls
 * each agent's step through the `self.schedule.step()`
 * the datacollector to collect each step `self.datacollector.collect(self)`

In [4]:
class GeoSchelling(mesa.Model):
    """Model class for the Schelling segregation model."""

    def __init__(self, density=0.6, minority_pc=0.2, export_data=False):
        super().__init__()
        self.density = density
        self.minority_pc = minority_pc
        self.export_data = export_data

        self.schedule = mesa.time.RandomActivation(self)
        self.space = mg.GeoSpace(crs="EPSG:4326", warn_crs_conversion=True)

        self.happy = 0
        self.datacollector = mesa.DataCollector({"happy": "happy"})

        self.running = True

        # Set up the grid with patches for every NUTS region
        ac = mg.AgentCreator(SchellingAgent, model=self)
        agents = ac.from_file("data/nuts_rg_60M_2013_lvl_2.geojson")
        self.space.add_agents(agents)

        # Set up agents
        for agent in agents:
            if random.random() < self.density:
                if random.random() < self.minority_pc:
                    agent.atype = 1
                else:
                    agent.atype = 0
                self.schedule.add(agent)

    def export_agents_to_file(self) -> None:
        self.space.get_agents_as_GeoDataFrame(agent_cls=SchellingAgent).to_crs(
            "epsg:4326"
        ).to_file("data/schelling_agents.geojson", driver="GeoJSON")

    def step(self):
        """Run one step of the model.

        If All agents are happy, halt the model.
        """

        self.happy = 0  # Reset counter of happy agents
        self.schedule.step()
        self.datacollector.collect(self)

        if self.happy == self.schedule.get_agent_count():
            self.running = False

        if not self.running and self.export_data:
            self.export_agents_to_file()

## Build the Visual

This section of code set ups the conditions for drawing the agents and defining the model parameters. The next cell passes the `schelling_draw` function and model parameters into Mesa Geo's `GeoJupyterViz` so we can see a visual for our simulation.  

#explanatory

Mesa-Geo's `GeoJupyterviz`requires users to pass in a function through `agent_portrayal` for agent visualization. This function then allows the agent to updates its representation based on user settings. In this case, the agent's color updates a dictionary that determines its color with the key "color" and the value being the agents color based on their affiliation (minority, majority, or none). 

`model_params` is then a dictionary with the parameters for the geoschelling model. `GeoSchelling` then takes three parameters and these parameters use Mesa's `UserParam` module to create tools users can employ to set model parameters. 
1. "density" is a slider tool ("SliderFloat") that takes a value from 0.0 to 1.0 with a step or accuracy at one tenth
2. "minority_pc" is also a slider tool ("SliderFloat") that the minority percent which takes a value of 0.0 to 1.0 with an accuracy of 5- 100th of a percent
3. "export_data" is then a binary tool ("checkbox") that allows user to turn the value to True by checking the book and exporting the data

In [5]:
def schelling_draw(agent):
    """
    Portrayal Method for canvas
    """
    portrayal = {}
    if agent.atype is None:
        portrayal["color"] = "Grey"
    elif agent.atype == 0:
        portrayal["color"] = "Orange"
    else:
        portrayal["color"] = "Blue"
    return portrayal


model_params = {
    "density": {
        "type": "SliderFloat",
        "value": 0.6,
        "label": "Population Density",
        "min": 0.0,
        "max": 0.9, #there must be an empty space for the agent to move
        "step": 0.1,
    },
    "minority_pc": {
        "type": "SliderFloat",
        "value": 0.2,
        "label": "Fraction Minority",
        "min": 0.0,
        "max": 1.0,
        "step": 0.05,
    },
    "export_data": {
        "type": "Checkbox",
        "value": False,
        "description": "Export Data",
        "disabled": False,
    },
}

## Run the Model

#explanatory

This cell then uses the previous cell to create a visualization of the model. In this case there are seven parameters. Two parameters being required. 

1. model: this passes in the model class which in this case is the `GeoSchelling` class model.
2. model_params: this takes the dictionary we built in the previous cell and passes in the parameters with the model types
3. measures: this is a kwarg (key word argument) which is not required but takes a list of measures users want to observe in this case the happy metric
4. name: also a kwarg for the name which shows in the model bar
5. agent_portrayal: a kwarg that allows us to pass in the function `schelling_draw` which determines agent representation based on their specific condition
6. zoom: a kwarg for the mapping function which determines what level the user should zoom on the map and takes an integer
7. center_point: a kwarg for telling the visualization what center point to use when portraying the map    

In [6]:
page = mgv.GeoJupyterViz(
    GeoSchelling,
    model_params,
    measures=["happy"],
    name="Geo-Schelling Model",
    agent_portrayal=schelling_draw,
    zoom=3,
    center_point=[52, 12],
)
# This is required to render the visualization in the Jupyter notebook
page

Cannot show ipywidgets in text